In [ ]:
import os
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

genai.configure(api_key=userdata.get('chave'))
titulo = "Hotel Central"
texto = "Hotel Central está localizado na região central da cidade de Rio Preto, com fácil acesso a supermercados, areas comerciais e de lazer"
model = "models/text-embedding-004"
embedding = genai.embed_content (title= titulo, content=texto, model=model, task_type="retrieval_document")
print(embedding)
print(len(embedding['embedding']))

{'embedding': [0.017913988, 0.0034170265, -0.025411295, -0.020095648, 0.0147040365, 0.021310182, 0.011122937, -0.03138722, 0.0063901353, 0.08830418, 0.015916755, 0.07240894, 0.0072495784, 0.021183945, 0.021678802, 0.01682837, -0.0065917275, 0.00012001617, -0.10472958, 0.011758058, 0.026816921, -0.040303852, 0.048364196, -0.009806201, 0.0044393777, 0.010436012, 0.028569577, 0.017107503, -0.08529738, -0.037419777, -0.009067026, 0.039799493, -0.06053356, -0.027050145, -0.0027882915, 0.034859043, 0.023652041, -0.06470702, 0.018653726, -0.042441927, 0.03345685, 0.034693137, -0.018583447, -0.028094077, -0.035468727, -0.10613288, -0.0054333694, 0.043817647, -0.0883944, 0.007973254, 0.009933907, -0.0011121014, -0.0536865, 0.036345962, -0.0043484233, -0.037721675, -0.02926897, -0.028173447, 0.02005038, 0.0075851204, 0.044979215, 0.017023917, -0.018005064, -0.0053023524, 0.052458126, -0.06987532, 0.0040487773, -0.0038988928, -0.033159237, -0.010003863, -0.035637435, 0.01634192, -0.019521128, -0.

In [ ]:
#!pip install gspread pandas
import gspread
import pandas as pd

csv = '/content/drive/MyDrive/Colab Notebooks/PLN_IA2/dados2_pln_sem_acento.csv'
df = pd.read_csv(csv, encoding="iso-8859-1", sep=";") # Se o separador for ponto e vírgula

# remover colunas NaN
df = df[['Titulo', 'Conteudo']]
df.head(10)

,Titulo,Conteudo
0,Hotel Central,"Nosso hotel oferece uma experiancia unica, com..."
1,Missao Hotel Central,Atender e servir com excelencia todos os hospe...
2,Localizacao do Hotel Central,"Praca Dom Udarico Rechiel, Centro de Rio Preto..."
3,Acomodacoes do Hotel Central,Desfrute de nossas suítes espacosas e bem deco...
4,Gastronomia do Hotel Central,Pratos deliciosos e generosos com a nossa gast...
5,Lazer e Recreacao do Hotel Central,"Salao de jogos, fliperama e piscina panoramica"
6,Eventos e Conferencias do Hotel Central,Nossos espaços para eventos sao perfeitos para...
7,Ofertas Especiais do Hotel Central,Confira nossas promocoes exclusivas para estad...
8,Servicos de Quarto do Hotel Central,"Disponivel 24 horas, nosso serviço de quarto o..."
9,Depoimentos de Hospedes do Hotel Central,Nossos hospedes amam a experiencia que proporc...


In [ ]:
df = df.dropna()

In [ ]:
df.head(10)

,Titulo,Conteudo
0,Hotel Central,"Nosso hotel oferece uma experiancia unica, com..."
1,Missao Hotel Central,Atender e servir com excelencia todos os hospe...
2,Localizacao do Hotel Central,"Praca Dom Udarico Rechiel, Centro de Rio Preto..."
3,Acomodacoes do Hotel Central,Desfrute de nossas suítes espacosas e bem deco...
4,Gastronomia do Hotel Central,Pratos deliciosos e generosos com a nossa gast...
5,Lazer e Recreacao do Hotel Central,"Salao de jogos, fliperama e piscina panoramica"
6,Eventos e Conferencias do Hotel Central,Nossos espaços para eventos sao perfeitos para...
7,Ofertas Especiais do Hotel Central,Confira nossas promocoes exclusivas para estad...
8,Servicos de Quarto do Hotel Central,"Disponivel 24 horas, nosso serviço de quarto o..."
9,Depoimentos de Hospedes do Hotel Central,Nossos hospedes amam a experiencia que proporc...


In [1]:
model = 'models/text-embedding-004'
def gerarEmbeddings(title, text):
  result = genai.embed_content(model=model,
                                content=text,
                                task_type="retrieval_document",
                                title=title)
  return result['embedding']

def gerarBuscarConsulta(consulta,dataset):
  embedding_consulta = genai.embed_content(model=model,
                                content=consulta,
                                task_type="retrieval_query",
                                )
  produtos_escalares = np.dot(np.stack(dataset["Embeddings"]), embedding_consulta['embedding']) # Calculo de distancia entre consulta e a base
  #maior_escalar = np.argmax(produtos_escalares)
  #print(embedding_consulta)
  print(produtos_escalares)
  indice = np.argmax(produtos_escalares)
  #print(produtos_escalares[indice])
  return dataset.iloc[indice]['Conteudo']

In [ ]:
df["Embeddings"] = df.apply(lambda row: gerarEmbeddings(row["Titulo"],row["Conteudo"]), axis=1)
print(df)

                                     Titulo  \
0                             Hotel Central   
1                      Missao Hotel Central   
2              Localizacao do Hotel Central   
3              Acomodacoes do Hotel Central   
4              Gastronomia do Hotel Central   
5        Lazer e Recreacao do Hotel Central   
6   Eventos e Conferencias do Hotel Central   
7        Ofertas Especiais do Hotel Central   
8       Servicos de Quarto do Hotel Central   
9  Depoimentos de Hospedes do Hotel Central   

                                            Conteudo  \
0  Nosso hotel oferece uma experiancia unica, com...   
1  Atender e servir com excelencia todos os hospe...   
2  Praca Dom Udarico Rechiel, Centro de Rio Preto...   
3  Desfrute de nossas suítes espacosas e bem deco...   
4  Pratos deliciosos e generosos com a nossa gast...   
5     Salao de jogos, fliperama e piscina panoramica   
6  Nossos espaços para eventos sao perfeitos para...   
7  Confira nossas promocoes exclus

In [ ]:
consulta = "Acomodacoes?"
resposta = gerarBuscarConsulta(consulta, df)
print(resposta)

[0.53288506 0.57255685 0.47264479 0.64669892 0.50407056 0.47929293
 0.54035834 0.53244332 0.52507539 0.55358785]
Desfrute de nossas suítes espacosas e bem decoradas, equipadas com todas as comodidades modernas para garantir o seu conforto.


In [ ]:
consulta = "Acomodacoes Hotel Central?"
resposta = gerarBuscarConsulta(consulta, df)
print(resposta)

[0.77129869 0.69598646 0.72912496 0.79815051 0.70883896 0.67798584
 0.74819758 0.75599414 0.68281996 0.74143472]
Desfrute de nossas suítes espacosas e bem decoradas, equipadas com todas as comodidades modernas para garantir o seu conforto.


In [ ]:
consulta = "Gastronomia Hotel Central?"
resposta = gerarBuscarConsulta(consulta, df)
print(resposta)

[0.66678983 0.60605446 0.62833566 0.60180847 0.7846849  0.59586941
 0.65001617 0.65334778 0.64488693 0.62711719]
Pratos deliciosos e generosos com a nossa gastronomia local


In [ ]:
consulta = "Lazer e recreacao do Hotel Central?"
resposta = gerarBuscarConsulta(consulta, df)
print(resposta)

[0.68118228 0.58676153 0.64179861 0.62199104 0.62332143 0.84751408
 0.6691801  0.64269745 0.60378464 0.63196556]
Salao de jogos, fliperama e piscina panoramica


In [ ]:
consulta = "Eventos e conferencias Hotel Central?"
resposta = gerarBuscarConsulta(consulta, df)
print(resposta)

[0.7085974  0.60354242 0.65373397 0.65922471 0.65661896 0.63908553
 0.84937164 0.70752277 0.62885866 0.66719111]
Nossos espaços para eventos sao perfeitos para conferencias, casamentos e celebracoes, com equipe dedicada para cuidar de cada detalhe.


In [ ]:
consulta = "Qual é a missao?"
resposta = gerarBuscarConsulta(consulta, df)
print(resposta)

[0.42859281 0.61849762 0.4296077  0.37303935 0.42269532 0.42320932
 0.43233584 0.39746238 0.39511443 0.40699024]
Atender e servir com excelencia todos os hospedes em um ambiente acongegante e familiar


In [ ]:
consulta = "Bem vindo Hotel Central?"
resposta = gerarBuscarConsulta(consulta, df)
print(resposta)

[0.75634359 0.67426192 0.72621811 0.70073116 0.71112557 0.67677561
 0.71584647 0.72381401 0.64355207 0.71993742]
Nosso hotel oferece uma experiancia unica, combinando luxo, conforto e um atendimento personalizado para tornar sua estadia inesquecivel.


In [ ]:
import pickle
pickle.dump(df, open('datasetEmbeddings.pkl','wb'))
